In [4]:
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, LSTM
from keras.layers import TimeDistributed
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import numpy as np
import itertools
import glob
import os
import json
import sys
import time
import math

In [7]:
def get_model():
    model = Sequential([
        TimeDistributed(Conv1D(16,3, activation='relu', padding = "same"),input_shape=[10,33,2]),
        TimeDistributed(BatchNormalization()),
        #TimeDistributed(MaxPooling1D()),
        TimeDistributed(Dropout(0.5)),
        #TimeDistributed(Conv1D(64,3, activation='relu',padding = "same")),
        BatchNormalization(),
        #TimeDistributed(Dropout(0.8)),
        TimeDistributed(Flatten()),
        #TimeDistributed(Dense(30,activation='softmax')),  
        LSTM(10,unit_forget_bias = 0.5, return_sequences = True),
        TimeDistributed(Dense(6,activation='softmax'))        
    ])
    adam = Adam(lr=0.0001)
    model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
    return model

In [8]:
asanas = {0:'bhujangasan', 1:'padamasan', 2:'shavasan', 3:'tadasan', 4:'trikonasan', 5:'vrikshasan'}

model = get_model()
model.load_weights("weights/val1-67-0.9971.hdf5")

In [9]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians= np.arctan2(c[1]-b[1],  c[0]-b[0])- np.arctan2(a[1]-b[1],  a[0]-b[0])
    angle = np.abs(radians*180/np.pi)

    if(angle>180):
        angle=360-angle
    
    return angle
    
def calculate_len(a,b):
    a = np.array(a)  # First point
    b = np.array(b)  # Last point
    r_sq = (a[0]-b[0])**2 + (a[1]-b[1])**2
    if(len(a)==3):
        r_sq = r_sq+(a[2]-b[2])**2
    r=math.sqrt(r_sq)
    return r


In [ ]:
# def landmark2list(landmark):
#     converted = [landmark.x,landmark.y]
#     return converted

In [10]:
def addToPred(Xch,count,Xarr,Xlist,seq,m):
    # print('Hello1')
    Xarr[count] = Xch[m]
    # print(Xarr.shape)
    # print(Xarr[count][11])
    # display_angles= calculate_angles_landmark(Xarr,count)
    # angles= []
    # angles.append(display_angles)
    # print(angles)
    # print('\n*****************\n')
    count = count + 1
    m=m+1
    # print(m,count)
    if count == 10:
        # print(display_angles)
        Xtemp = []
        Xtemp.append(Xarr)
        Xlist[seq] = Xarr
        Xarr = np.empty((10,33,2))
        count = 0
        seq  = seq + 1
        #printing prediction for first frame        
        Yht_pred = model.predict(np.asarray(Xtemp))
        Y_pred = np.argmax(Yht_pred[0],axis = 1)
        Y_pred_asanas = np.array([asanas[a] for a in Y_pred])
        print("Prediction " + " is " + str(Y_pred_asanas[0]))
        
    # print(angles)
    return (seq, count, Xarr,m)

In [11]:
def makePred(filename,count,Xarr,Xlist,seq,m):
    with open(filename) as json_data:
        #     print('1')
            d = json.load(json_data)    
            X = np.asarray(d)
            X1 =  X[:,:,0]
            X2 = X[:,:,1]
            Xch = np.dstack((X1,X2))
            nof= Xch.shape[0]
            seq ,count, Xarr,m = addToPred(Xch,count,Xarr,Xlist,seq,m)
            json_data.close
        #     print('4')
    return (seq, count, Xarr,m, nof)

In [19]:
count = 0
seq = 0
Xarr = np.empty((10,33,2))
Xlist = np.empty((1000,10,33,2))
#print(Xarr.shape)
lastRead = "none"
m=0

# correct_file = 'Correct_asana/intermediate_vriksh2_photo.json'

while(True):
    #print(count)
    list_of_files = glob.glob("output/15_Veena_Shav.json")
    latest_file = max(list_of_files, key=os.path.getctime)
    if lastRead != latest_file:
        try:
            seq, count, Xarr,m, nof = makePred(latest_file,count,Xarr,Xlist,seq,m)
            # compare_angles(correct_file, latest_file)
        except:
            print ("Frame issue")
    if m==nof:
        break

1/1 [==============================] - 0s 17ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 17ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 20ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 22ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 22ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 22ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 58ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 20ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 18ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 18ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 20ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 17ms/step
Prediction  is shavasan
